# product_and_consumer
执行一次发送和接收


# 创建连接


In [1]:
import pika

In [2]:
connection_params = pika.ConnectionParameters(
    host="127.0.0.1",
    port=5672,
    virtual_host="/",
    credentials=pika.PlainCredentials("admin", "123456"),
)

rabbit_proudct_connection = pika.BlockingConnection(connection_params)

# 声明队列


## queue_declare

参数

1. queue：很简单，队列的名字。
2. passive：默认 False，声明队列时，队列不存在会自动创建；指定为 True，那么队列不存在时会报错。
3. durable：持久化，注意这里的持久化指的是队列的持久化
4. exclusive：个人觉得该参数实现的功能有点奇怪，不明白它的意义何在，这里不做介绍。
5. auto_delete：当队列中的消息都被消费完毕、并且最后一个消费者断开连接时，是否自动删除该队列
6. arguments：一个字典，用于接收一些额外的参数，主要针对于更高级特性


In [3]:
QUEUE_FOR_TEST_NAME = "boys"

In [4]:
proudct_channel = rabbit_proudct_connection.channel()
proudct_channel.queue_declare(queue=QUEUE_FOR_TEST_NAME, durable=True)

<METHOD(['channel_number=1', 'frame_type=1', "method=<Queue.DeclareOk(['consumer_count=0', 'message_count=0', 'queue=boys'])>"])>

# 发送消息


In [5]:
# 不指定，就会发送到默认exchange
# delivery_mode=2 让消息持久化，1 则是非持久化
proudct_channel.basic_publish(
    exchange="", routing_key=QUEUE_FOR_TEST_NAME, body=b"my name is kagura_mea",properties=pika.BasicProperties(delivery_mode=2)
)

proudct_channel.close()
rabbit_proudct_connection.close()

# 消费消息


## 创建消费者连接

In [6]:
connection_params = pika.ConnectionParameters(
    host="127.0.0.1",
    port=5672,
    virtual_host="/",
    credentials=pika.PlainCredentials("admin", "123456"))

rabbit_consumer_connection = pika.BlockingConnection(connection_params)
consumer_channel = rabbit_consumer_connection.channel()
# must
consumer_channel.queue_declare(queue=QUEUE_FOR_TEST_NAME, durable=True)


<METHOD(['channel_number=1', 'frame_type=1', "method=<Queue.DeclareOk(['consumer_count=0', 'message_count=1', 'queue=boys'])>"])>

## 定义消息接收回调参数


### 自动ACK

In [7]:
from pprint import pprint


def on_message_callback(ch, method, properties, body):
    pprint({"ch": ch, "method": method, "properties": properties, "body": body})

### 手动ACK

In [8]:
# 手动Ack版本的回调函数
def on_message_manual_ack_callback(ch, method, properties, body):
    print("消息处理······")
    ch.basic_ack(delivery_tag=method.delivery_tag)

## 阻塞消费消息
basic_consume的参数
1. 队列的名字
2. on_message_callback 表示回调函数，消息发送给消费者后，会拆解成四个参数自动传递给回调函数，进行调用。
3.  auto_ack 表示是否自动应答，收到消息后，是否应答已收到
4. exclusive 表示是否独占该队列，默认为 False。如果消费者在监听时指定为 True，那么其它消费者就无法再监听该队列了，当然指定为 True 的前提是该队列目前还没有被其它消费者监听。
5. consumer_tag 表示给消费者指定的标签，如果不指定会默认生成一个。
6. arguments 表示指定的高级特性，一个字典，暂时先不讨论。


In [9]:
# 订阅消息
consumer_channel.basic_consume(QUEUE_FOR_TEST_NAME, 
                      on_message_callback=on_message_callback,
                      auto_ack=True)
# 会阻塞一直监听队列、持续等待消息
consumer_channel.start_consuming()

{'body': b'my name is kagura_mea',
 'ch': <BlockingChannel impl=<Channel number=1 OPEN conn=<SelectConnection OPEN transport=<pika.adapters.utils.io_services_utils._AsyncPlaintextTransport object at 0x000002426DECE2C0> params=<ConnectionParameters host=127.0.0.1 port=5672 virtual_host=/ ssl=False>>>>,
 'method': <Basic.Deliver(['consumer_tag=ctag1.42c0c7f967f1490eb398aa210f194670', 'delivery_tag=1', 'exchange=', 'redelivered=False', 'routing_key=boys'])>,
 'properties': <BasicProperties(['delivery_mode=2'])>}
